In [1]:
# Set your API key to a variable
import os
openai_api_key = os.environ["OPENAI_API_KEY"]

# Import the required packages
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [2]:
# Load the HTML as a LangChain document loader
loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()

In [3]:
# Instantiate RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Maximum size of each chunk
    chunk_overlap=200  # Overlap between chunks
)

Split text from car_docs

In [4]:
# Split the text from car_docs into chunks
car_doc_chunks = text_splitter.split_documents(car_docs)

# Display the number of chunks created
print(len(car_doc_chunks))
print(car_doc_chunks)


9
[Document(metadata={'source': 'data/mg-zs-warning-messages.html'}, page_content='Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.'), Document(metadata={'source': 'data/mg-zs-warning-messages.html'}, page_content='Warning Message Procedure Low Oil Pressure Indicate

Use Chroma to store chunks

Now, let's store the document chunks in a Chroma vector store. Chroma is a vector database that can be used to store and query embeddings efficiently.

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Create a Chroma vector store from the document chunks
vector_store = Chroma.from_documents(car_doc_chunks, embeddings, persist_directory="chroma_db")

# Display the number of vectors stored
len(vector_store)

C:\Users\Trung Thao\AppData\Local\Temp\ipykernel_20044\1169415459.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


27

Create a retriever

Now that we have stored the document chunks in a Chroma vector store, we can create a retriever. A retriever allows us to query the vector store to find the most relevant document chunks for a given query.

In [6]:
# Create a retriever from the Chroma vector store
retriever = vector_store.as_retriever()

# # Example query to test the retriever
query = "What is the process of car manufacturing?"

# # Retrieve the most relevant document chunks
relevant_chunks = retriever.get_relevant_documents(query)

# # Display the retrieved chunks
for chunk in relevant_chunks:
    print(chunk)

C:\Users\Trung Thao\AppData\Local\Temp\ipykernel_20044\24109770.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_chunks = retriever.get_relevant_documents(query)


page_content='Warning Message Procedure Stop Start System Fault Indicates that the Stop/Start intelligent fuel saving system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Clutch Switch Fault Indicates that the clutch switch has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Gasoline Particular Filter Full Indicates that the gasoline particular filter is full. Please consult an MG Authorised Repairer as soon as possible. Ignition System Fault Indicates that the ignition system has detected a fault. Please consult an MG Authorised Repairer immediately.' metadata={'source': 'data/mg-zs-warning-messages.html'}
page_content='Warning Message Procedure Stop Start System Fault Indicates that the Stop/Start intelligent fuel saving system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Clutch Switch Fault Indicates that the clutch switch has detected a fault. Please consult an MG Authorised

Initialize ChatOpenAI with model=gpt-4o-mini and temperature=0

In [7]:
from langchain.chat_models import ChatOpenAI

# Initialize the ChatOpenAI model with specified parameters
chat_model = ChatOpenAI(model='gpt-4o-mini', temperature=0, openai_api_key=openai_api_key)

C:\Users\Trung Thao\AppData\Local\Temp\ipykernel_20044\3041841681.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model='gpt-4o-mini', temperature=0, openai_api_key=openai_api_key)


Create ChatPromptTemplate with the context variable

### Create a ChatPromptTemplate

To interact with the `ChatOpenAI` model, we need to create a `ChatPromptTemplate`. This template will define the structure of the prompts we send to the model, including any context or variables that should be included.

In [8]:
from langchain.prompts import PromptTemplate

# Define a context variable
prompt = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

# Create a ChatPromptTemplate with the context variable
prompt_template = PromptTemplate(
    template=prompt
)

# Example usage of the prompt template
user_input = "How does the car manufacturing process work?"
formatted_prompt = prompt_template.invoke({
    "question": user_input,
    "context": "Fake context"
})

print(formatted_prompt)

text="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: How does the car manufacturing process work? \nContext: Fake context \nAnswer:\n"


In [9]:
# Setup the chain
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt_template
    | chat_model
)

In [10]:
# Initialize query
query = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"

# Invoke the query
answer = rag_chain.invoke(query).content
print(answer)

The "Gasoline Particular Filter Full" warning means that the gasoline particulate filter is full and needs attention. You should consult an MG Authorised Repairer as soon as possible to address this issue.
